# Iris classification problem with TensorFlow

In [7]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os.path

In [19]:
TRAIN_URL = "http://download.tensorflow.org/data/iris_training.csv"
TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"
DATA_DIR = "./data/"

CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

train_file = os.path.abspath(DATA_DIR + TRAIN_URL.split('/')[-1]) # /.../.../iris_training.csv
test_file = os.path.abspath(DATA_DIR + TEST_URL.split('/')[-1]) # /.../../iris_test.csv

download the data files if they are not there and save them to current ./data subdirectory.

In [20]:
def maybe_download():
    tf.keras.utils.get_file(train_file, TRAIN_URL)
    tf.keras.utils.get_file(test_file, TEST_URL)

In [8]:
def load_data(y_name='Species'):
    """Returns the iris dataset as (train_x, train_y), (test_x, test_y)."""
    maybe_download()

    train = pd.read_csv(train_file, names=CSV_COLUMN_NAMES, header=0)
    train_x, train_y = train, train.pop(y_name)                                 

    test = pd.read_csv(test_file, names=CSV_COLUMN_NAMES, header=0)
    test_x, test_y = test, test.pop(y_name)

    return (train_x, train_y), (test_x, test_y)

In [10]:
(train_x, train_y), (test_x, test_y) = load_data()

In [20]:
train_x.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [17]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [18]:
test_x.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,5.9,3.0,4.2,1.5
1,6.9,3.1,5.4,2.1
2,5.1,3.3,1.7,0.5
3,6.0,3.4,4.5,1.6
4,5.5,2.5,4.0,1.3


In [19]:
test_y.head()

0    1
1    2
2    0
3    1
4    1
Name: Species, dtype: int64

In [23]:
train_x.keys()

Index(['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth'], dtype='object')

In [24]:
# Feature columns describe how to use the input.                            
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [27]:
my_feature_columns

[_NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 _NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]